In [1]:
import os
import matplotlib.pyplot as plt
os.chdir('./data/UNOSAT/Syria/')
from glob import glob
shp_file=glob('Homs/*.shp')

FileNotFoundError: [Errno 2] No such file or directory: './data/UNOSAT/Syria/'

# 转换边界文件

In [ ]:
pd=gpd.read_file(glob('Hama/1_Set*.shp')[0])
pd=pd.to_crs(4326)
idxs=[5,2,0,23,1,3]
name=[pd.iloc[i]['Name'] for i in idxs]
name=[i.replace(' ','-') for i in name]
for i in idxs:
    s_sub=pd.iloc[i]
    pd_sub=gpd.GeoDataFrame({'Name':s_sub.Name,'Source':s_sub.Source,'geometry':s_sub.geometry},index=[0])
    pd_sub.to_file(s_sub.Name+'_settlements.shp')

# 转换点文件

In [ ]:
def get_pandas(pd,id):
    c=['SiteID','SensDt','DmgCls','StlmtNme','geometry']
    c2=c.copy()
    if id>1.2:
        c2[1]+=f'_{str(id)}'
        c2[2]+=f'_{str(id)}'
    pd_sub=pd[pd[c2[2]].notnull()]
    pd_sub_c=pd_sub.reindex(columns=c2)
    pd_sub_c=pd_sub_c.rename(columns={c2[1]:c[1]})    
    pd_sub_c=pd_sub_c.rename(columns={c2[2]:c[2]})
    print(len(pd_sub))
    return pd_sub_c
k=['A','B','C','D','E']
for n in name:
    n=n.replace('-','_')
    f=glob(f'Hama/*Damage_Sites_{n}*.shp')[0]
    pd=gpd.read_file(f)
    pd=pd.to_crs(4326)
    dates_n=np.array([int(i[-1]) for i in list(pd.columns) if 'SensID' in i ]).max()
    print(dates_n)
    for date in range(1,dates_n+1):
        pd_sub=get_pandas(pd,date)
        d=list(pd_sub['SensDt'])[0]
        print(d)
        n_=n.replace('_','-')
        f_name=f'{n_}_{k[date]}_{d}.shp'
        print(f_name)
        pd_sub.to_file(f_name)

# 转换无标注区域

In [ ]:
for n in name:
    n=n.replace('-','_')
    f=glob(f'Hama/*No*{n}*.shp')[0]
    pd=gpd.read_file(f)
    pd=pd.to_crs(4326)
    n_=n.replace('_','-')
    filename=f'{n_}_NoAnalysis.shp'
    print(filename)
    pd.to_file(filename)
pd=gpd.read_file('Hama/3_Cloud_Cover_Aleppo_20140523.shp')
pd=pd.to_crs(4326)
pd.to_file('Aleppo_C_cloud.shp')
pd=gpd.read_file('Hama/3_Cloud_Cover_Idlib_20140502.shp')
pd=pd.to_crs(4326)
pd.to_file('Idlib_C_cloud.shp')